# 01 - Data-I/O and featurisation

In this notebook, we will cover how to load (and visualize) molecular simulation data.

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import mdshare
import pyemma

## Case 1: preprocessed data (toy model)
In the most convenient case, we already have preprocessed data available in some kind of archive which can be read using `numpy`:

In [ ]:
file = mdshare.load('hmm-doublewell-2d-100k.npz', working_directory='data')
with np.load(file) as fh:
    data = fh['trajectory']

print(data)

Once we have the data in memory, we can use one of `pyemma`'s plotting functions to visualize what we have loaded:

In [ ]:
pyemma.plots.plot_feature_histograms(data, feature_labels=['$x$', '$y$'])

The `plot_feature_histograms()` function visualizes the distributions of all degrees of freedom where we assume that the columns of `data` represent different features and the rows represent different time steps.

While `plot_feature_histograms()` can handle arbitrary nhmbers of features, we have an additional plotting function for the special case of two features:

In [ ]:
fig, ax = pyemma.plots.plot_free_energy(*data.T)
ax.set_xlabel('$x$')
ax.set_ylabel('$y$')
fig.tight_layout()

The `plot_free_energy()` function makes a two-dimensional histogram for the given features and visualizes their free energy surface.

## Case 2: loading `*.dcd` files (alanine dipeptide)
To load molecular dynamics data from one of the standard file formats, we need not only the actual simulation data, but a topology file, too.

In [ ]:
pdb = mdshare.load('alanine-dipeptide-nowater.pdb', working_directory='data')
files = [mdshare.load('alanine-dipeptide-%d-250ns-nowater.dcd' % i, working_directory='data') for i in range(3)]
print(pdb)
print(files)

We start with creating a featurizer object using the topology file:

In [ ]:
feat = pyemma.coordinates.featurizer(pdb)

Next, we start adding features which we want to extract from the simulation data. Here, we want to load the backbone torsions:

In [ ]:
feat.add_backbone_torsions()

We can always call the featurizer's `describe()` method to show which features are requested:

In [ ]:
print(feat.describe())

After we have selected all desired features, we can call the `load()` function to load all features into memory or, alternatively, the `source()` function to create a streamed feature reader. For now, we will use `load()`:

In [ ]:
data = pyemma.coordinates.load(files, features=feat)

print(data)

Apparently, we have loaded a list of three two-dimensional `numpy.ndarray` objects from our three trajectory files. We can visualize these features using the aforementioned plotting functions, but to do so we have to concatenate the three individual trajectories:

In [ ]:
pyemma.plots.plot_feature_histograms(np.concatenate(data), feature_labels=['$\Phi$', '$\Psi$'])

In [ ]:
fig, ax = pyemma.plots.plot_free_energy(*np.concatenate(data).T)
ax.set_xlabel('$\Phi$')
ax.set_ylabel('$\Psi$')
fig.tight_layout()

Let us look at a different featurization example and load the positopns of all heavy atoms instead. We create a new featurizer object and use its `add_selection()` method to request the positions of a given selection of atoms. For this selctuion, we can use the `select_Heavy()` method which returns the indices of all heavy atoms.

Again, we load the data into memory and show what we loaded using the `describe()` method:

In [ ]:
feat = pyemma.coordinates.featurizer(pdb)
feat.add_selection(feat.select_Heavy())

data = pyemma.coordinates.load(files, features=feat)

feat_desc = feat.describe()
print(feat_desc)

And we visualize the distributions of all loaded features:

In [ ]:
fig, ax = plt.subplots(figsize=(10, 7))
pyemma.plots.plot_feature_histograms(np.concatenate(data), feature_labels=feat_desc, ax=ax)
fig.tight_layout()

### `load()` versus `source()`
Using `load()`, we put the full data into memory which is possible for all examples in this tutorial:

In [ ]:
print(data)

Many real world apllications, though, require more memory than your workstation might provide. For these cases, you should use the `source()` function:

In [ ]:
data = pyemma.coordinates.source(files, features=feat)

print(data)

This function allows to stream the data and work on chunks instead of the full set. Most of the functions in the `coordinate` submodule accept data in memory as well as streamed feature readers but some plotting functions can only work with data in memory. To load a (strided) subset into memory, we can use the `get_output()` method with a stride parameter:

In [ ]:
data_out = data.get_output(stride=5)

print(data_out)

We now have loaded every fifth time step into memory and we can visualize the (concatenated) features:

In [ ]:
fig, ax = plt.subplots(figsize=(10, 7))
pyemma.plots.plot_feature_histograms(np.concatenate(data_out), feature_labels=feat_desc, ax=ax)
fig.tight_layout()

### Exercise: heavy atom distances
Please fix the following code block such that the distances between all heavy atoms are loaded and visualized.

**Hint**: you might find the `add_distances()` method of the featuriuzer object helpful.

In [ ]:
feat = pyemma.coordinates.featurizer(pdb)
feat. #FIXME

data = pyemma.coordinates.load(files, features=feat)

fig, ax = plt.subplots(figsize=(10, 7))
pyemma.plots.plot_feature_histograms(np.concatenate(data), feature_labels=feat.describe(), ax=ax)
fig.tight_layout()

In [ ]:
feat = pyemma.coordinates.featurizer(pdb)
feat.add_distances(feat.select_Heavy())

data = pyemma.coordinates.load(files, features=feat)

fig, ax = plt.subplots(figsize=(10, 7))
pyemma.plots.plot_feature_histograms(np.concatenate(data), feature_labels=feat.describe(), ax=ax)
fig.tight_layout()

## Case 3: loading `*.xtc` files (pentapeptide)
The handling of `*.xtc` files is identical to that of `*.dcd` files or any other standard molecular dynamics fiel format supported by `pyemma`'s dependency `mdtraj`. ONce we have obtained the raw data files...

In [ ]:
pdb = mdshare.load('pentapeptide-impl-solv.pdb', working_directory='data')
files = [mdshare.load('pentapeptide-%02d-500ns-impl-solv.xtc' % i, working_directory='data') for i in range(25)]

... we can load a selection of features into memory. Here, we want the $\cos/\sin$ transformations of the backbone and $\chi_1$ sidechain torsions...

In [ ]:
feat = pyemma.coordinates.featurizer(pdb)
feat.add_backbone_torsions(cossin=True)
feat.add_sidechain_torsions(which='chi1', cossin=True)

data = pyemma.coordinates.load(files, features=feat)

feat_desc = feat.describe()
print(feat_desc)

... and finally, we visualize the (concatenated) features:

In [ ]:
fig, ax = plt.subplots(figsize=(10, 7))
pyemma.plots.plot_feature_histograms(np.concatenate(data), feature_labels=feat_desc, ax=ax)
fig.tight_layout()

### Exercises: feature selection and visualization

**Exercise 1**: complete the following code block to load/visualize the distances between all $\text{C}_\alpha$ carbon atoms.

**Hint**: you might find the `add_distances_ca()` method of the featurizer object helpful.

In [ ]:
feat = pyemma.coordinates.featurizer(pdb)
feat. #FIXME

data = pyemma.coordinates.load(files, features=feat)

fig, ax = plt.subplots(figsize=(10, 7))
pyemma.plots.plot_feature_histograms(np.concatenate(data), feature_labels=feat.describe(), ax=ax)
fig.tight_layout()

In [ ]:
feat = pyemma.coordinates.featurizer(pdb)
feat.add_distances_ca()

data = pyemma.coordinates.load(files, features=feat)

fig, ax = plt.subplots(figsize=(10, 7))
pyemma.plots.plot_feature_histograms(np.concatenate(data), feature_labels=feat.describe(), ax=ax)
fig.tight_layout()

**Exercise 2**: complete the following code block to load/visualize the minimal distances between all residues.

**Hint**: you might find the `add_residue_mindist()` method of the featurizer object helpful.

In [ ]:
feat = pyemma.coordinates.featurizer(pdb)
feat. #FIXME

data = pyemma.coordinates.load(files, features=feat)

fig, ax = plt.subplots(figsize=(10, 7))
pyemma.plots.plot_feature_histograms(np.concatenate(data), feature_labels=feat.describe(), ax=ax)
fig.tight_layout()

In [ ]:
feat = pyemma.coordinates.featurizer(pdb)
feat.add_residue_mindist()

data = pyemma.coordinates.load(files, features=feat)

fig, ax = plt.subplots(figsize=(10, 7))
pyemma.plots.plot_feature_histograms(np.concatenate(data), feature_labels=feat.describe(), ax=ax)
fig.tight_layout()

**Exercise 3**: complete the following code block to load/visualize the position of all backbone atoms.

**Hint**: you might find the `select_Backbone()` method of the featurizer object helpful.

In [ ]:
feat = pyemma.coordinates.featurizer(pdb)
feat. #FIXME

data = pyemma.coordinates.load(files, features=feat)

fig, ax = plt.subplots(figsize=(10, 7))
pyemma.plots.plot_feature_histograms(np.concatenate(data), feature_labels=feat.describe(), ax=ax)
fig.tight_layout()

In [ ]:
feat = pyemma.coordinates.featurizer(pdb)
feat.add_selection(feat.select_Backbone())

data = pyemma.coordinates.load(files, features=feat)

fig, ax = plt.subplots(figsize=(10, 7))
pyemma.plots.plot_feature_histograms(np.concatenate(data), feature_labels=feat.describe(), ax=ax)
fig.tight_layout()

**Exercise 4**: complete the following code block to load/visualize the position of all $\text{C}_\alpha$ atoms.

**Hint**: you might find the `select_Ca()` method of the featurizer object helpful.

In [ ]:
feat = pyemma.coordinates.featurizer(pdb)
feat. #FIXME

data = pyemma.coordinates.load(files, features=feat)

fig, ax = plt.subplots(figsize=(10, 7))
pyemma.plots.plot_feature_histograms(np.concatenate(data), feature_labels=feat.describe(), ax=ax)
fig.tight_layout()

In [ ]:
feat = pyemma.coordinates.featurizer(pdb)
feat.add_selection(feat.select_Ca())

data = pyemma.coordinates.load(files, features=feat)

fig, ax = plt.subplots(figsize=(10, 7))
pyemma.plots.plot_feature_histograms(np.concatenate(data), feature_labels=feat.describe(), ax=ax)
fig.tight_layout()

## Wrapping up
In this notebook, we have learned how to load and visualize molecular simulation data with `pyemma`. In detail, we have used
- `pyemma.coordinates.featurizer()` to define a selection of features we want to extract,
- `pyemma.coordinates.load()` to load data into memory, and
- `pyemma.coordinates.source()` to create a streamed feature reader in case the data does not fit into memory.

Once have have data in memory, we have used

- `pyemma.plots.plot_feature_histograms()` to show the distributions of all loaded features and
- `pyemma.plots.plot_free_energy()` to visualize the free energy surface of two selected features.